In [1]:
import pandas as pd

In [2]:
def read_vcf(vcf_path):
    df = pd.read_csv(vcf_path, comment='#', sep='\t', header=None, names=['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE'])
    return df

In [10]:
# I need to make sure the nist vcf file has END column. The SVs are not visualized correctly without END in IGV.
nist_sv_hg38_vcf = read_vcf('/Users/gaoyueya/Documents/Projects/TAG-Ticket1639/HG002_hg38/dragen4_2_NIST/hg38/HG002_GRCh38_CMRG_SV_v1.00.vcf')
nist_sv_hg38_vcf.head()

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE
0,chr1,1041776,.,CCGGCCAGTGCCAGGGTCGAGGTGGGCGGCTCCCCCGGGGGAGGGCTG,C,30,.,REPTYPE=CONTRAC;BREAKSIMLENGTH=143;REFWIDENED=...,GT:AD,"1|1:0,2"
1,chr1,23884345,.,T,TTAGAGGCGTGAGCCACTACACCTGGCTATGTATACTTTAGTGATT...,30,.,REPTYPE=SIMPLEINS;BREAKSIMLENGTH=1;REFWIDENED=...,GT:AD,"1|1:0,2"
2,chr1,25405592,.,T,TGCAATGAGCTATGATTGTACCACTGGGAAGTGACAAAGGGCACCC...,30,.,REPTYPE=SIMPLEDEL;BREAKSIMLENGTH=8393;REFWIDEN...,GT:AD,"1|1:0,2"
3,chr1,115686862,.,GTTGTATCAATGAGGATAATAATAATATCTCTCTGACAGGGTTGTT...,G,30,.,REPTYPE=SIMPLEDEL;BREAKSIMLENGTH=3;REFWIDENED=...,GT:AD,"1|0:1,1"
4,chr1,204195224,.,T,TACACACACACACCACACACATATACACACATACACCCATACACAC...,30,.,REPTYPE=SIMPLEINS;BREAKSIMLENGTH=184;REFWIDENE...,GT:AD,"1|0:0,1"


In [19]:
set([row['INFO'].split(';')[0].split(',')[0].split('REPTYPE=')[1] for index, row in nist_sv_hg38_vcf.iterrows()])

{'CONTRAC', 'DUP', 'SIMPLEDEL', 'SIMPLEINS', 'SUBSDEL'}

In [23]:
# Add the END attribute that is required by IGV visualization
updated_info_field = []
for index, row in nist_sv_hg38_vcf.iterrows():
    info_list = row['INFO'].split(';')
    svlen=None
    svtype=None
    # Get SVLEN and Assign END
    if len(row['ALT']) > len(row['REF']):
        svlen = len(row['ALT'])
        svtype = 'INS'
        
    elif len(row['ALT']) < len(row['REF']):
        svlen = len(row['REF'])
        svtype = 'DEL'
    end_pos = row['POS'] + svlen
    
    # Extend the original info list with three new elements
    info_list.extend([f"END={end_pos}", f"SVLEN={svlen}", f"SVTYPE={svtype}"])
    # Join the list
    updated_info = ';'.join(info_list)
    updated_info_field.append(updated_info)
updated_info_field

['REPTYPE=CONTRAC;BREAKSIMLENGTH=143;REFWIDENED=chr1:1041679-1041868;END=1041824,;SVLEN=48,;SVTYPE=DEL;END=1041824;SVLEN=48;SVTYPE=DEL',
 'REPTYPE=SIMPLEINS;BREAKSIMLENGTH=1;REFWIDENED=chr1:23884346-23884346;END=23886497,;SVLEN=2152,;SVTYPE=INS;END=23886497;SVLEN=2152;SVTYPE=INS',
 'REPTYPE=SIMPLEDEL;BREAKSIMLENGTH=8393;REFWIDENED=chr1:25415381-25405617;END=25405702,;SVLEN=110,;SVTYPE=INS;END=25405702;SVLEN=110;SVTYPE=INS',
 'REPTYPE=SIMPLEDEL;BREAKSIMLENGTH=3;REFWIDENED=chr1:115686863-115690222;END=115690220,;SVLEN=3358,;SVTYPE=DEL;END=115690220;SVLEN=3358;SVTYPE=DEL',
 'REPTYPE=SIMPLEINS;BREAKSIMLENGTH=184;REFWIDENED=chr1:204195135-204195318;END=204195606,;SVLEN=382,;SVTYPE=INS;END=204195606;SVLEN=382;SVTYPE=INS',
 'REPTYPE=SIMPLEINS;BREAKSIMLENGTH=95;REFWIDENED=chr1:204195135-204195229;END=204195680,;SVLEN=456,;SVTYPE=INS;END=204195680;SVLEN=456;SVTYPE=INS',
 'REPTYPE=DUP;BREAKSIMLENGTH=459;REFWIDENED=chr2:982922-983380;END=983488,;SVLEN=223,;SVTYPE=INS;END=983488;SVLEN=223;SVTYPE=I

In [24]:
# Update INFO field
nist_sv_hg38_vcf['INFO'] = updated_info_field

In [26]:
nist_sv_hg38_vcf.to_csv('/Users/gaoyueya/Documents/Projects/TAG-Ticket1639/HG002_hg38/dragen4_2_NIST/hg38/HG002_GRCh38_CMRG_SV_v1.00_igv.vcf', sep='\t', index=False, header=False)

In [7]:
nist_sv_hg38_vcf

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE
0,1,10403,HG2_Ill_svaba_2,ACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAAC,A,10,LongReadHomRef,ClusterIDs=HG4_Ill_GATKHC_1:HG4_Ill_svaba_2:HG...,GT:GTcons1:PB_GT:PB_REF:PB_ALT:PBHP_GT:PB_REF_...,./.:./.:./.:1:0:./.:0:0:0:0:./.:17:5:28:3:./.:...
1,1,10415,HG3_Ill_GATKHC_3,ACCCTAACCCTAACCCTAACCCTAAC,A,10,LongReadHomRef,ClusterIDs=HG2_Ill_GATKHC_2:HG3_Ill_svaba_2:HG...,GT:GTcons1:PB_GT:PB_REF:PB_ALT:PBHP_GT:PB_REF_...,./.:./.:./.:0:1:./.:0:0:0:0:./.:10:7:16:5:./.:...
2,1,10616,HG4_Ill_GATKHC_3,CCGCCGTTGCAAAGGCGCGCCG,C,10,NoConsensusGT,ClusterIDs=HG2_Ill_150bpfermikitraw_1:HG4_Ill_...,GT:GTcons1:PB_GT:PB_REF:PB_ALT:PBHP_GT:PB_REF_...,./.:./.:./.:0:4:./.:0:0:0:0:./.:0:0:0:0:0/1:6:...
3,1,54712,HG2_Ill_250bpfermikitraw_2,T,TTTTTTTTTCTTTCTTTCTTTCTTTCTTTCTTTCTTTCTTTCTTTC...,10,NoConsensusGT,ClusterIDs=HG2_Ill_250bpfermikitraw_2:HG2_Ill_...,GT:GTcons1:PB_GT:PB_REF:PB_ALT:PBHP_GT:PB_REF_...,./.:./.:./.:2:3:./.:0:0:0:0:./.:1:2:0:0:1/1:0:...
4,1,385977,HG2_PB_PB10Xdip_2154,A,CGTGTTTGTCCTGATGGTCTCCTACCCCCTGTCCCCTGAGAGGCCC...,10,NoConsensusGT,ClusterIDs=HG2_PB_PB10Xdip_2154:HG2_PB_PB10Xdi...,GT:GTcons1:PB_GT:PB_REF:PB_ALT:PBHP_GT:PB_REF_...,./.:./.:./.:0:1:./.:0:0:0:0:./.:0:0:0:0:./.:0:...
...,...,...,...,...,...,...,...,...,...,...
74007,Y,59002942,HG4_PB_HySA_29505,T,TAATATTAAAATTTCAATCAACAGTCACAAAAGCAGACTAATAAAG...,20,NoConsensusGT,ClusterIDs=HG4_PB_HySA_29505:HG3_Ill_Spiral_20...,GT:GTcons1:PB_GT:PB_REF:PB_ALT:PBHP_GT:PB_REF_...,./.:./.:./.:32:9:./.:0:0:0:0:./.:0:0:0:0:0/0:2...
74008,Y,59004477,HG2_Ill_GATKHCSBGrefine_13224,AAGAAAGAAAGAAAGAAAGAAAGAAAGAAAGAAAGAAAGAAAGAAA...,A,20,NoConsensusGT,ClusterIDs=HG2_PB_pbsv_20599:HG4_PB_HySA_29508...,GT:GTcons1:PB_GT:PB_REF:PB_ALT:PBHP_GT:PB_REF_...,./.:./.:0/1:33:21:./.:0:0:0:0:./.:0:0:0:0:0/1:...
74009,Y,59013741,HG3_PB_SVrefine2PBcRDovetail_10185,T,TCCTGCCTCAGCCTCCCAAGTAGCTGGGATTACAGGCACCTGCCGC...,10,NoConsensusGT,ClusterIDs=HG2_PB_SVrefine2Falcon1plusDovetail...,GT:GTcons1:PB_GT:PB_REF:PB_ALT:PBHP_GT:PB_REF_...,./.:./.:0/1:25:40:./.:0:0:0:0:./.:0:0:0:0:0/1:...
74010,Y,59019531,HG2_Ill_scalpel_18829,CACACACACACACACACACACAA,C,10,NoConsensusGT,ClusterIDs=HG2_Ill_150bpfermikitraw_24951:HG4_...,GT:GTcons1:PB_GT:PB_REF:PB_ALT:PBHP_GT:PB_REF_...,./.:./.:0/1:62:32:./.:0:0:0:0:./.:0:0:0:0:0/1:...
